In [2]:
import numpy as np

from math import log

def entropy(ele):
    ele=ele.tolist()
    probs=[ele.count(i)/len(ele) for i in set(ele)]
    
    entropy=-sum([prob*log(prob,2) for prob in probs])
    
    return entropy

In [3]:
import pandas as pd

df=pd.read_csv('example_data.csv')
df.head()

,humility,outlook,play,temp,windy
0,high,sunny,no,hot,False
1,high,sunny,no,hot,True
2,high,overcast,yes,hot,False
3,high,rainy,yes,mild,False
4,normal,rainy,yes,cool,False


In [5]:
entropy_D=entropy(df['play'])
entropy_D

0.9402859586706309

In [6]:
def df_split(df,col):
    unique_col_val=df[col].unique()
    res_dict={elem:pd.DataFrame for elem in unique_col_val}
    for key in res_dict.keys():
        res_dict[key]=df[df[col]==key]
    return res_dict

In [7]:
x=df_split(df,'temp')
x['hot']

,humility,outlook,play,temp,windy
0,high,sunny,no,hot,False
1,high,sunny,no,hot,True
2,high,overcast,yes,hot,False
12,normal,overcast,yes,hot,False


In [46]:
def choose_best_feature(df,label):
    entropy_D=entropy(df[label])
    
    cols=[col for col in df.columns]
    cols.remove(label)
    max_value=-999
    best_feature,max_splited=None,None
    
    for col in cols:
        splited_set=df_split(df,col)
        entropy_DA=0
        
        for subset_col,subset in splited_set.items():
            entropy_Di=entropy(subset[label])
            entropy_DA+=len(subset)/len(df)*entropy_Di
        
        info_gain=entropy_D-entropy_DA
        
        if info_gain>max_value:
            max_value,best_feature=info_gain,col
            max_splited=splited_set
    
    return max_value,best_feature,max_splited

In [47]:
# ID3算法类
class ID3Tree:
    # 定义决策树结点类
    class TreeNode:
        # 定义树结点
        def __init__(self, name):
            self.name = name
            self.connections = {}
        # 定义树连接
        def connect(self, label, node):
            self.connections[label] = node
    # 定义全局变量，包括数据集、特征集、标签和根结点
    def __init__(self, df, label):
        self.columns = df.columns
        self.df = df
        self.label = label
        self.root = self.TreeNode("Root")
    # 构建树的调用
    def construct_tree(self):
        self.construct(self.root, "", self.df, self.columns)
    # 决策树构建方法
    def construct(self, parent_node, parent_label, sub_df, columns):
        # 选择最优特征
        max_value, best_feature, max_splited = choose_best_feature(sub_df[columns], self.label)
        # 如果选不到最优特征，则构造单结点树
        if not best_feature:
            node = self.TreeNode(sub_df[self.label].iloc[0])
            parent_node.connect(parent_label, node)
            return
        # 根据最优特征以及子结点构建树
        node = self.TreeNode(best_feature)
        parent_node.connect(parent_label, node)
        # 以A-Ag为新的特征集
        new_columns = [col for col in columns if col != best_feature]
        # 递归地构造决策树
        for splited_value, splited_data in max_splited.items():
            self.construct(node, splited_value, splited_data, new_columns)

In [48]:
# 创建ID3决策树实例
id3_tree = ID3Tree(df, 'play')
# 构造ID3决策树
id3_tree.construct_tree()

['humility', 'outlook', 'temp', 'windy']